# GGG基盤上に製膜したやつの再学習

In [ ]:
#------------------------
# Google Colab上でのみ実行
#------------------------
import time
%env TOKEN=*************************************
! git clone https://$$TOKEN@github.com/konnitiha3/MOD2NN.git

import sys
sys.path.append('/content/MOD2NN')

from google.colab import drive
drive.mount('/content/drive')

!pip install GPyOpt

In [ ]:
import time
import os
import json
import sys
import re
import glob
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import GPyOpt

from Faraday.two_dim.module.lib.layers import *
from Faraday.two_dim.module.lib import regularizer

tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-5000:]
y_val = y_train[-5000:]
x_train = x_train[:-5000]
y_train = y_train[:-5000]

In [1]:
#@title デフォルトのタイトル テキスト
wavelength = 532.0e-9 #@param {type:"number"}
d = 1.0e-6 #@param {type:"number"}
n = 1.5 #@param {type:"number"}

| wavelength(nm) | FR(deg.)     | E(deg.)     |
|----------------|--------------|-------------|
| 532            | -2.104965093 | 0.978522465 |

In [1]:
def create_model(**kwargs):
    z = kwargs["z"]
    r1 = kwargs["r1"]
    r2 = kwargs["r2"]
    wavelength = 532.0e-9
    d = 1.0e-6
    n = 1.5
    #tf.random.set_seed(kwargs["seed"])
    shape = (100, 100)
    inputs = tf.keras.Input((28, 28))
    theta = -2.105 * np.pi / 180
    eta = np.arctan(0.9785 * np.pi/180)/2
    l1=1.0e-5
    print(kwargs)
    x = ImageResizing(shape)(inputs)
    x = ImageBinarization(0.5, 0.0, 1.0)(x)
    x = IntensityToElectricField(shape)(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta, kernel_regularizer=regularizer.ShiftL1Regularizer(l1, np.pi/2))(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.5e-3, d=d, n=1.51, method='expand')(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta, kernel_regularizer=regularizer.ShiftL1Regularizer(l1, np.pi/2))(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=1.00, method='expand')(x)
    # x = Polarizer(shape)(x)
    #x =ElectricFieldToIntensity(shape)(x)
    #x = MNISTFilter(shape)(x)
    x = FaradayRotationByStokes(shape)(x)
    # x = Argument(shape)(x)
    #x = MNISTDetector(10)(x)
    x = CircleOnCircumferenceDetector(10, r1, r2)(x)
    x = tf.keras.layers.Softmax()(x)
    model = tf.keras.Model(inputs, x)
    return model

In [ ]:
history = []
count = 0
def train(**kwargs):
    global count
    count += 1
    model = create_model(**kwargs)
    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,  # category: sparse_categorical_crossentropy
                  metrics=['accuracy'])

    epochs = 50
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='accuracy',
        min_delta=0.05,
        patience=2,
    )

    model_name = "20220808_BO/" + str(count)
    cholab_path = "/content/drive/MyDrive/D2NN/"
    checkpoint_path = cholab_path + "checkpoint/" + model_name + "/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # チェックポイントコールバックを作る
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1)

    logdir = os.path.join(cholab_path +"logs", model_name)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    result = model.fit(x_train,
                       y_train,
                       batch_size=64,
                       epochs=epochs,
                       validation_data=(x_val, y_val),
                       callbacks=[cp_callback, tensorboard_callback]
                       )

    path = cholab_path + "trained_model/"+ model_name
    model.save(path)

    df = pd.DataFrame(result.history)
    df.to_csv(path + "/history.csv")

    with open(path + "/config.json", 'w') as f:
        json.dump(model.get_config(), f, indent=4)

    bi_model = tf.keras.models.load_model(path)
    pattern = r'mo'
    mo_layers = []
    for layer in bi_model.layers:
        result = re.match(pattern, layer.name)
        if result:
            mo_layers.append(layer)

    for layer in mo_layers:
        w = layer.get_weights()
        bi_w = np.where(w[0]>0, np.pi/2, -np.pi/2)
        w[0] = bi_w
        layer.set_weights(w)
    evaluate = bi_model.evaluate(x_test, y_test)
    history.append([count, *kwargs.values(), evaluate[0], evaluate[1]])
    df = pd.DataFrame(history)
    df.to_csv("/content/drive/MyDrive/D2NN/trained_model/20220808_BO/bo_history.csv")

    return evaluate

In [ ]:
bounds = [{'name': 'z',  'type': 'continuous',  'domain': (0.1e-3, 5.0e-3)},
          {'name': 'r1', 'type': 'discrete',  'domain': range(1,50)},
          {'name': 'r2', 'type': 'discrete',  'domain': range(1,50)}]

constraints = [
    {
        "name": "constr_1",
        "constraint": "(x[:,1] + x[:,2]) - 50" # r1 + r2 <= 50
    },
    {
        "name": "constr_2",
        "constraint": "x[:,1]*0.325 - x[:,2]" # r2 >= r1*tan(2pi/10)
    }
]

# ベイズ最適化する関数（上記で書いたブラックボックス）を定義します。
# xが入力で、出力はreturnされます。
def f(x):
    print(x)
    evaluation = train(
        z = float(x[:,0]),
        r1 = float(x[:,1]),
        r2 = int(x[:,2]))
    print("loss:{0} \t\t accuracy:{1}".format(evaluation[0], evaluation[1]))
    print(evaluation)
    return evaluation[0]

# 事前探索を行います。
opt_mnist = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds, constraints=constraints)

# 最適なパラメータを探索します。
opt_mnist.run_optimization(max_iter=50)
print("optimized parameters: {0}".format(opt_mnist.x_opt))
print("optimized loss: {0}".format(opt_mnist.fx_opt))